In [ ]:
/* Demo notebook for FOSDEM2019
   Tested with beakerx-1.3.0 on python-3.6.7
*/
%classpath add jar target/fosdem2019-0.0.1-SNAPSHOT.jar
%%classpath add mvn
    org.apache.tinkerpop tinkergraph-gremlin 3.2.6

In [ ]:
import nl.vtslab.fosdem2019.traversal.AuthorizedTraversalSource
import static org.apache.tinkerpop.gremlin.structure.VertexProperty.Cardinality.set;
import org.apache.tinkerpop.gremlin.tinkergraph.structure.TinkerGraph

src1Authz = "8;-;-";
src2Authz = "8;2;-";
src3Authz = "9;3;11";
src4Authz = "9;4;11";
src5Authz = "9;5;11";

def createGraph() {
    graph = TinkerGraph.open();
    g = graph.traversal();
    g.addV("Person").property("name", "p0").property("authz", src1Authz).property(set, "authz", src2Authz).next();
    g.addV("Person").property("name", "p1").property("authz", src3Authz).next();
    g.addV("Person").property("name", "p2").property("authz", src4Authz).next();
    g.addV("Event").property("name", "v1").property("authz", src1Authz).next();
    g.addV("Event").property("name", "v2").property("authz", src4Authz).next();
    g.V().has("name", "v1").as("a").
            V().has("name", "p0").addE("Visits").to("a").property("name", "e01").property("authz", src1Authz).
            V().has("name", "p1").addE("Visits").to("a").property("name", "e11").property("authz", src3Authz).next();
    g.V().has("name", "v2").as("a").
            V().has("name", "p1").addE("Visits").to("a").property("name", "e12").property("authz", src3Authz).
            V().has("name", "p2").addE("Visits").to("a").property("name", "e22").property("authz", src5Authz).next();
    return graph;
}

graph = createGraph();
g = graph.traversal();
ga = {x -> graph.traversal(AuthorizedTraversalSource.class)}
"graph and traversal sources available"

## Normal operation

In [ ]:
unrestricted = g.V().values("name").toList()
authorized = ga().withAuthorization([src1Authz]).V().values("name").toList()
"unrestricted: " + unrestricted + "\nauthorized: " + authorized